In [ ]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

from data import result_loader
from training import plotter

# Experiment evaluation
This interactive notebook loads specs, histories and models from files generated by a past experiment and plots the results.

The next cell prints all specs-files from the subdirectory `/results` and asks for the experiment to load.
Simply copy a complete line into the text field and press enter to perform the evaluation.

In [ ]:
result_loader.print_relative_spec_file_paths()
specs_file = input("\nPlease choose a specs-file: ")
experiment_prefix = result_loader.extract_experiment_path_prefix(specs_file)
absolute_specs_path = result_loader.generate_absolute_specs_path(specs_file)

In [ ]:
# load experiment's specs and results
specs = result_loader.get_specs_from_file(absolute_specs_path)
loss_hists, val_acc_hists, test_acc_hists, val_acc_hists_epoch, test_acc_hists_epoch, sparsity_hist = result_loader.get_histories_from_file(experiment_prefix)
nets = result_loader.get_models_from_files(experiment_prefix, specs)

In [ ]:
specs

In [ ]:
plotter.plot_average_val_acc_and_loss(val_acc_hists_epoch, loss_hists, sparsity_hist, specs['plot_step'])

In [ ]:
plotter.plot_average_acc(test_acc_hists_epoch, sparsity_hist, test=True)

In [ ]:
plotter.plot_acc_at_early_stopping(test_acc_hists, loss_hists, sparsity_hist, specs['plot_step'])

In [ ]:
# compare final test-accuracies
for n in range(specs['net_count']):
    unpruned_final = test_acc_hists_epoch[n,0,-1]
    for p in range(1, specs['prune_count']+1):
        pruned_final = test_acc_hists_epoch[n,p,-1]
        print(f"Test-accuracy net #{n} (sparsity: {sparsity_hist[p]:.4}): {pruned_final:1.4} (changed by {(pruned_final-unpruned_final):1.4})")

In [ ]:
nets[0].sparsity_report()